In [5]:
import numpy as np
from matplotlib import pyplot as plt
import math as m

In [6]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display


def step_slice(lst, step):
    return lst[step]


def animate_list(lst, play=False, interval=200):
    slider = widgets.IntSlider(min=0, max=len(lst) - 1, step=1, value=0)
    if play:
        play_widjet = widgets.Play(interval=interval, max=len(lst) - 1)
        widgets.jslink((play_widjet, 'value'), (slider, 'value'))
        display(play_widjet)
    return interact(step_slice,
                    lst=fixed(lst),
                    step=slider)

In [7]:
#2*u_tt = u_xx + ch(x-t)
#u(x,0) = ch(x)
#u_t(x,0) = -sh(x)
#u(0,t)-u_x(0,t) = e^t
#2*u(1,t) -u_x(1,t) = 0.5(e^(1-t)+3*e^(t-1))
#u_real = ch(x-t)
# wave equation
# u_tt = c*u_xx + f(x,t)
# x [lft,rgt]
# t > 0
# u(x,0) = f_0(x)
# u_t(x,0) = f_0t(x)
# b0*u(lft,t) + a0*u_x(lft,t) = f_lft(t)
# b1*u(rgt,t) + a1*u_x(rgt,t) = f_rgt(t)

In [8]:
speed = 1/2 #a^2

a0, b0, a1, b1 = -1, 1, -1, 2



def u_real(x,t):
    return np.array([m.cosh(xk-t) for xk in x])

def f(x,t):
    return m.cosh(x-t)/2

#начальные условия
def phi1(x):
    return  m.cosh(x)

def phi2(x):
    return -m.sinh(x)

#граничные условия
def gamma1(t):
    return m.e**t

def gamma2(t):
    return (m.e**(1-t)+3*(m.e**(t-1)))/2

x_min =  0
x_max = 1
h_cur = 0.01
h_cur_ = h_cur / 2

t_min = 0
t_max = 1
t_cur = 0.005
t_cur_ = t_cur / 2

In [9]:
def cross(step,h,t,gamma1,gamma2,U1,a0,b0,a1,b1,scheme = '1'):
    
    U = [ speed*((t / h)**2)*(U1[1][k+1] - 2 * U1[1][k] + U1[1][k-1]) + (t**2)*f(h*k, t*step-t) + 2*U1[1][k] - U1[0][k]
         for k in range(1, len(U1[0]) - 1)]
    
    if(scheme == '1'):
        U = [(gamma1(t*step) - (a0/h)*U[0]) * (h/(b0*h-a0))] + U + [(gamma2(t*step)+(a1/h)*U[-1])*(h/(b1*h+a1))]
        
    elif(scheme == '2'):

        U = [(gamma1(t*step)*h - 2*a0*U[0] + 0.5*a0*U[1]) / (b0*h - 1.5*a0)] + U \
        + [(gamma2(t*step)*h + 2*a1*U[-1] - 0.5*a1*U[-2]) / (b1*h + 1.5*a1)]

        
    else:
        raise (Exception('Неправильный порядок точности'))

    
    return [U1[1].copy(),U.copy()]

    

In [10]:
def phi1_xx(x):
    return  m.cosh(x)

#аппроксимация начальных условий, h = 10^-2
tn_cur = np.arange(t_min, t_max + t_cur, t_cur)
xk_cur = np.arange(x_min, x_max + h_cur, h_cur)
#1 порядок,  h = 10^-2
u1 = [[],[]]
u1[0] = [phi1(i) for i in xk_cur]
u1[1] = [phi2(i) * t_cur + phi1(i) for i in xk_cur]
#2 порядок, h = 10^-2
u2 = [[],[]]
u2[0] = [phi1(i) for i in xk_cur]
u2[1] = [phi1(i) + t_cur * phi2(i) + (t_cur**2) * 0.5 * (speed * phi1_xx(i) + f(i, 0)) for i in xk_cur]


#аппроксимация начальных условий, h = 5*10^-3
tn_cur_ = np.arange(t_min, t_max + t_cur_, t_cur_)
xk_cur_ = np.arange(x_min, x_max + h_cur_, h_cur_)
#1 порядок
u1_ = [[],[]]
u1_[0] = [phi1(i) for i in xk_cur_]
u1_[1] = [phi2(i) * t_cur_ + phi1(i) for i in xk_cur_]
#2 порядок, h = 10^-2
u2_ = [[],[]]
u2_[0] = [phi1(i) for i in xk_cur_]
u2_[1] = [phi1(i) + t_cur_ * phi2(i) + (t_cur_**2) * 0.5 * (speed * phi1_xx(i) + f(i, 0)) for i in xk_cur_]


In [11]:
plots= []

err1 =[m.sqrt(np.mean(np.square(u_real(xk_cur,0) - u1[0])))]
err2 =[m.sqrt(np.mean(np.square(u_real(xk_cur,0) - u2[0])))]

err1_ =[m.sqrt(np.mean(np.square(u_real(xk_cur_,0) - u1_[0])))]
err2_ =[m.sqrt(np.mean(np.square(u_real(xk_cur_,0) - u2_[0])))]

err1_max =[max(np.abs(u_real(xk_cur,0) - u1[0]))]
err2_max =[max(np.abs(u_real(xk_cur,0) - u2[0]))]

err1_max_ =[max(np.abs(u_real(xk_cur_,0) - u1_[0]))]
err2_max_ =[max(np.abs(u_real(xk_cur_,0) - u2_[0]))]

fig, ax = plt.subplots(3, 1, figsize=(15, 15))

ax[0].plot(xk_cur, u1[0], '--r', label="h = 10^-2, 1 порядок")
ax[0].plot(xk_cur, u2[0], '--b', label="h = 10^-2, 2 порядок")

ax[0].plot(xk_cur_, u1_[0], '--y', label="h = 5*10^-3, 1 порядок")
ax[0].plot(xk_cur_, u2_[0], '--c', label="h = 5*10^-3, 2 порядок")

ax[0].plot(xk_cur_, u_real(xk_cur_,0), 'g', label="real")

ax[0].legend()
ax[0].set_xlabel('xk')
ax[0].set_ylabel('u')
ax[0].set_title('0.000')


ax[1].plot([0], err1, 'r', label="h = 10^-2, mse, 1 порядок")
ax[2].plot([0], err2, '--r', label="h = 10^-2, mse, 2 порядок")

ax[1].plot([0], err1_, 'y', label="h = 5*10^-3, mse, 1 порядок")
ax[2].plot([0], err2_, '--y', label="h = 5*10^-3, mse, 2 порядок")

ax[1].plot([0], err1_max, 'm', label="h = 10^-2, max, 1 порядок")
ax[2].plot([0], err2_max, '--m', label="h = 10^-2, max, 2 порядок")

ax[1].plot([0], err1_max_, 'c', label="h = 5*10^-3, max, 1 порядок")
ax[2].plot([0], err2_max_, '--c', label="h = 5*10^-3, max, 2 порядок")

ax[1].legend()
ax[1].set_title('Погрешности')
ax[1].set_xlabel('Время, t')

ax[2].legend()
ax[2].set_title('Погрешности')
ax[2].set_xlabel('Время, t')

plt.close(fig)
plots.append(fig)

In [12]:


err1.append(m.sqrt(np.mean(np.square(u_real(xk_cur,t_cur) - u1[1]))))
err2.append(m.sqrt(np.mean(np.square(u_real(xk_cur,t_cur) - u2[1]))))

err1_.append(m.sqrt(np.mean(np.square(u_real(xk_cur_,t_cur_) - u1_[1]))))
err2_.append(m.sqrt(np.mean(np.square(u_real(xk_cur_,t_cur_) - u2_[1]))))

err1_max.append(max(np.abs(u_real(xk_cur,t_cur) - u1[1])))
err2_max.append(max(np.abs(u_real(xk_cur,t_cur) - u2[1])))

err1_max_.append(max(np.abs(u_real(xk_cur_,t_cur_) - u1_[1])))
err2_max_.append(max(np.abs(u_real(xk_cur_,t_cur_) - u2_[1])))

fig, ax = plt.subplots(3, 1, figsize=(15, 15))

ax[0].plot(xk_cur, u1[1], '--r', label="h = 10^-2, 1 порядок")
ax[0].plot(xk_cur, u2[1], '--b', label="h = 10^-2, 2 порядок")

ax[0].plot(xk_cur_, u1_[1], '--y', label="h = 5*10^-3, 1 порядок")
ax[0].plot(xk_cur_, u2_[1], '--c', label="h = 5*10^-3, 2 порядок")

ax[0].plot(xk_cur_, u_real(xk_cur_,0), 'g', label="real")

ax[0].legend()
ax[0].set_xlabel('xk')
ax[0].set_ylabel('u')
ax[0].set_title('0.005')


ax[1].plot([0,t_cur], err1, 'r', label="h = 10^-2, mse, 1 порядок")
ax[2].plot([0,t_cur], err2, '--r', label="h = 10^-2, mse, 2 порядок")

ax[1].plot([0,t_cur_], err1_, 'y', label="h = 5*10^-3, mse, 1 порядок")
ax[2].plot([0,t_cur_], err2_, '--y', label="h = 5*10^-3, mse, 2 порядок")

ax[1].plot([0,t_cur], err1_max, 'm', label="h = 10^-2, max, 1 порядок")
ax[2].plot([0,t_cur], err2_max, '--m', label="h = 10^-2, max, 2 порядок")

ax[1].plot([0,t_cur_], err1_max_, 'c', label="h = 5*10^-3, max, 1 порядок")
ax[2].plot([0,t_cur_], err2_max_, '--c', label="h = 5*10^-3, max, 2 порядок")

ax[1].legend()
ax[1].set_title('Погрешности')
ax[1].set_xlabel('Время, t')

ax[2].legend()
ax[2].set_title('Погрешности')
ax[2].set_xlabel('Время, t')


plt.close(fig)
plots.append(fig)

In [13]:
%%time

for step in range(2, len(tn_cur_)):
    
    u1_ = cross(step,h_cur_,t_cur_,gamma1,gamma2,u1_,a0,b0,a1,b1) #5*10^-3, 1 порядок
    
    u2_ = cross(step,h_cur_,t_cur_,gamma1,gamma2,u2_,a0,b0,a1,b1,scheme = '2') #5*10^-3, 2 порядок

    
    if (step % 2 == 1):

        u1 = cross((step+1)/2,h_cur,t_cur,gamma1,gamma2,u1,a0,b0,a1,b1) #10^-2, 1 порядок

        u2 = cross((step+1)/2,h_cur,t_cur,gamma1,gamma2,u2,a0,b0,a1,b1,scheme = '2') #10^-2, 2 порядок


        err1_max.append(np.max(np.abs(u_real(xk_cur,t_cur*(step+1)/2) - u1[1])))
        err2_max.append(np.max(np.abs(u_real(xk_cur,t_cur*(step+1)/2) - u2[1])))
        
        err1_max_.append(np.max(np.abs(u_real(xk_cur_,t_cur_*step) - u1_[1])))
        err2_max_.append(np.max(np.abs(u_real(xk_cur_,t_cur_*step) - u2_[1])))

        err1.append(m.sqrt(np.mean(np.square(u_real(xk_cur,t_cur*(step+1)/2) - u1[1]))))
        err2.append(m.sqrt(np.mean(np.square(u_real(xk_cur,t_cur*(step+1)/2) - u2[1]))))
        
        err1_.append(m.sqrt(np.mean(np.square(u_real(xk_cur_,t_cur_*step) - u1_[1]))))
        err2_.append(m.sqrt(np.mean(np.square(u_real(xk_cur_,t_cur_*step) - u2_[1]))))
        
        fig, ax = plt.subplots(3, 1, figsize=(15, 15))

        ax[0].plot(xk_cur_, u_real(xk_cur_,t_cur_*step), 'g', label="real")
        
        ax[0].plot(xk_cur, u1[1], '--r', label="h = 10^-2, 1 порядок")
        ax[0].plot(xk_cur, u2[1], '--b', label="h = 10^-2, 2 порядок")
        
        ax[0].plot(xk_cur_, u1_[1], '--y', label="h = 5*10^-3, 1 порядок")
        ax[0].plot(xk_cur_, u2_[1], '--c', label="h = 5*10^-3, 2 порядок")
        
        ax[0].legend()
        ax[0].set_xlabel('xk')
        ax[0].set_ylabel('u')
        ax[0].set_title('{:.2f}'.format(step*t_cur_))
        

        ax[1].plot([i*t_cur_ for i in range(len(err1))], err1, 'r', label="h = 10^-2, mse, 1 порядок")
        ax[2].plot([i*t_cur_ for i in range(len(err2))], err2, '--r', label="h = 10^-2, mse, 2 порядок")
        
        ax[1].plot([i*t_cur_ for i in range(len(err1_))], err1_, 'y', label="h = 5*10^-3, mse, 1 порядок")
        ax[2].plot([i*t_cur_ for i in range(len(err2_))], err2_, '--y', label="h = 5*10^-3, mse, 2 порядок")
        
        ax[1].plot([i*t_cur_ for i in range(len(err1_max))], err1_max, 'm', label="h = 10^-2, max, 1 порядок")
        ax[2].plot([i*t_cur_ for i in range(len(err2_max))], err2_max, '--m', label="h = 10^-2, max, 2 порядок")
        
        ax[1].plot([i*t_cur_ for i in range(len(err1_max_))], err1_max_, 'c', label="h = 5*10^-3, max, 1 порядок")
        ax[2].plot([i*t_cur_ for i in range(len(err2_max_))], err2_max_, '--c', label="h = 5*10^-3, max, 2 порядок")

        
        ax[1].legend()
        ax[1].set_title('Погрешности')
        ax[1].set_xlabel('Время, t')
        
        ax[2].legend()
        ax[2].set_title('Погрешности')
        ax[2].set_xlabel('Время, t')

        
        plt.close(fig)
        plots.append(fig)
print('отношение max погрешностей mse для 1-го порядка:', max(err1)/max(err1_))
print('отношение max погрешностей max 1-го порядка:', max(err1_max)/max(err1_max_))
print('отношение max погрешностей mse 2-го порядка:', max(err2)/max(err2_))
print('отношение max погрешностей max 2-го порядка:', max(err2_max)/max(err2_max_))

отношение max погрешностей mse для 1-го порядка: 0.18334826927610587
отношение max погрешностей max 1-го порядка: 0.17581286383541198
отношение max погрешностей mse 2-го порядка: 0.03544592300408456
отношение max погрешностей max 2-го порядка: 0.051282718073278916
Wall time: 3.05 s


In [14]:
animate_list(plots, play=True, interval=100)

Play(value=0, max=50)

interactive(children=(IntSlider(value=0, description='step', max=50), Output()), _dom_classes=('widget-interac…

<function __main__.step_slice(lst, step)>